In [ ]:
!unzip images.zip # colab unzip images

In [ ]:
from models.models2D.cnn import CNN
from datasets.Images2D import ImageDataset
from utils.device_setter import device_setter

In [ ]:
DEFAULT_IMAGE_SIZE = 512 # Images have different sizes, so we have to resize them to common size
BATCH_SIZE = 64
EPOCHS = 10
TEST_SIZE = 0.25
LR_RATE = 0.01

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

IMAGE_DATA_PATH_POS = os.getenv('POS_PATH') if os.getenv('POS_PATH') is not None else "./data/png_out/pos"
IMAGE_DATA_PATH_NEG = os.getenv('NEG_PATH') if os.getenv('NEG_PATH') is not None else "./data/png_out/neg"

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((DEFAULT_IMAGE_SIZE, DEFAULT_IMAGE_SIZE))
])

In [ ]:
dataset = ImageDataset(IMAGE_DATA_PATH_POS, IMAGE_DATA_PATH_NEG, transform)

In [ ]:
print(f"Dataset len: {len(dataset)}")
print(f"Positives: {len(dataset.images_positves)}, Negatives: {len(dataset.images_negatives)}, Negatives ratio: {len(dataset.images_negatives) / len(dataset) * 100}")

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset


# labels are required here only for balancing dataset
train_idx, val_idx, _, _ = train_test_split(
    list(range(len(dataset))),
    dataset.targets,
    test_size=TEST_SIZE,
    shuffle=True,
    stratify=dataset.targets,
)

In [ ]:
datasets = {}
datasets["train"] = Subset(dataset, train_idx)
datasets["val"] = Subset(dataset, val_idx)

In [ ]:
device = device_setter()
device

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(datasets["train"], batch_size=BATCH_SIZE)
test_dataloader = DataLoader(datasets["val"], batch_size=BATCH_SIZE)

In [ ]:
import matplotlib.pyplot as plt

train_features, train_labels = next(iter(train_dataloader))

print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

img = train_features[0].squeeze()
label = train_labels[0]

plt.imshow(img, cmap="gray")
plt.show()

print(f"Label: {label}")

In [ ]:
model = CNN().to(device)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch

optimizer = optim.Adam(model.parameters(), lr=LR_RATE)

loss_criteria = nn.CrossEntropyLoss()
for epoch in range(1, EPOCHS + 1):
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data, target = data.float().to(device), target.to(device)
        optimizer.zero_grad()

        output = model(data)
        _, predicted = torch.max(output.data, 1)

        loss = loss_criteria(output, target)

        train_loss += loss.item()

        loss.backward()
        optimizer.step()

        print(
            f"\tTarget: {float(torch.sum(target) / len(target) * 100)}, Predict: {float(torch.sum(predicted) / len(target) * 100)} Training batch {batch_idx + 1} Loss: {loss.item():.6f}"
        )

    avg_loss = train_loss / (batch_idx + 1)
    print("Training set: Average loss: {:.6f}".format(avg_loss))
    print("-" * 40)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        batch_count = 0
        for data, target in test_dataloader:
            batch_count += 1
            data, target = data.float().to(device), target.to(device)

            output = model(data)

            test_loss += loss_criteria(output, target).item()

            _, predicted = torch.max(output.data, 1)
            print(
                f"Target: {float(torch.sum(target) / len(target) * 100)} Predict: {float(torch.sum(predicted) / len(target) * 100)}"
            )

            correct += torch.sum(target == predicted).item()

    avg_loss = test_loss / batch_count
    print(
        "Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            avg_loss,
            correct,
            len(test_dataloader.dataset),
            100.0 * correct / len(test_dataloader.dataset),
        )
    )